In [1]:
import numpy as np
import qgutils as qg
import sys

In [2]:
def get_psi(p, fo):
    
    '''
    parameters
    
    p : array[nz, nx, ny]
    f0 : scalar 
    
    '''
    
    nd = p.ndim
    
    if nd < 3:
        print("not handling less than 3D arrays")
        sys.exit(1)
        
    psi = qg.interp_on_c(p)/f0
    
    return psi


In [5]:
#moyenne sur le temps

'''
f : netcdf file
f0 : coriolis parameter
si_x, si_y, si_t : size of psi
'''

def mean_psi(f, f0, si_x, si_y, si_t):
    psi_me = 0.0*np.zeros((si_x,si_y))
    n_me = 0
    nt = 0
    for nt in range(0,si_t):
        psi = get_psi(f.variables['p'][nt,:,:,:].copy(),f0)
        psi_me = psi_me + psi
        n_me   = n_me + 1

    return psi_me/n_me
    

In [6]:
#moyenne sur le temps de KE psi turbulent

'''
f : netcdf file
f0 : coriolis parameter
si_x, si_y, si_t : size of psi
psi_me : moyenne temporelle de psi
'''

def mean_psi_turb(f1, f0, psi_me, si_x, si_y, si_t, Delta ):
    ke_turb_me  = 0.0*np.zeros((si_y,si_x))
    nt=0
    for nt in range(0,si_t):
        psi_tu = get_psi(f1.variables['p'][nt,:,:,:].copy(), f0) - psi_me
        ke_turb_me = ke_turb_me + qg.comp_ke(psi_tu, Delta) 
    
    return ke_turb_me/si_t

In [20]:
def psi_filtre (f, t_filtre):
    
    '''
    function which gives an iterator of the filtered dataset 
    
    parameters
    
    psi : [nt, 3, (,nx, ny)]
    others : scalars
    '''
    
    nd = psi.ndim
    
    if nd < 4:
        print("not handling less than 4D arrays")
        sys.exit(1)
    
    
    sha_nt = psi[:,0,0,0].shape
    if sha_nt[0]<= 2*t_filtre:
        print("number of time step isn't big enough to handle the filter")
        sys.exit(1)
    
    si_z, si_x, si_y = psi.shape
    
    psi_filt = np.zeros((si_t - 2*t_filtre, 3, si_x, si_y))
    nt = t_filtre
    
    for nt in range(t_filtre, si_t2 - t_filtre):
        psi_filt[nt-t_filtre,:,:,:] = np.mean(psi[nt-t_filtre:nt+t_filtre,:,:,:], axis=0)
        nt = nt+1
        
    yield psi_filt